In [17]:
import datetime

print(datetime.datetime.now())

2022-09-03 15:24:07.797323


In [102]:
df = pd.read_csv('atendimentos.csv')
df

,ID,Tipo_Senha,Numeracao,Data_Emissao,Guiche,Data_Atendimento
0,1,SP,1,2022-09-03 14:58:23.641908,1.0,2022-09-03 15:08:23.641908
1,2,SE,1,2022-09-03 14:58:23.641909,1.0,2022-09-03 15:18:23.641909
2,3,SG,1,2022-09-03 14:58:23.641910,2.0,2022-09-03 14:59:23.641910
3,4,SP,2,2022-09-03 14:58:23.641911,1.0,2022-09-03 15:28:23.641911
4,5,SE,2,2022-09-03 14:58:23.641912,2.0,2022-09-03 15:32:23.641912
5,6,SP,3,2022-09-03 14:58:23.641913,2.0,2022-09-03 15:40:23.641913
6,7,SG,2,2022-09-03 14:58:23.641914,3.0,2022-09-03 15:35:23.641914
7,8,SG,3,2022-09-03 14:58:23.641915,NaN,NaN
8,9,SP,4,2022-09-03 14:58:23.641916,NaN,NaN
9,10,SG,4,2022-09-02 14:58:23.641917,NaN,NaN


,ID,Tipo_Senha,Numeracao,Data_Emissao,Guiche,Data_Atendimento
2,3,SG,1,2022-09-03 14:58:23.641910,2.0,2022-09-03 14:59:23.641910
0,1,SP,1,2022-09-03 14:58:23.641908,1.0,2022-09-03 15:08:23.641908
1,2,SE,1,2022-09-03 14:58:23.641909,1.0,2022-09-03 15:18:23.641909
3,4,SP,2,2022-09-03 14:58:23.641911,1.0,2022-09-03 15:28:23.641911
4,5,SE,2,2022-09-03 14:58:23.641912,2.0,2022-09-03 15:32:23.641912
6,7,SG,2,2022-09-03 14:58:23.641914,3.0,2022-09-03 15:35:23.641914
5,6,SP,3,2022-09-03 14:58:23.641913,2.0,2022-09-03 15:40:23.641913


In [107]:
ultimo_chamado = df[~df["Data_Atendimento"].isna()].sort_values(by = "Data_Atendimento").tail(1)
ultimo_tipo_senha = ultimo_chamado["Tipo_Senha"].values[0]
ultimo_chamado

,ID,Tipo_Senha,Numeracao,Data_Emissao,Guiche,Data_Atendimento
5,6,SP,3,2022-09-03 14:58:23.641913,2.0,2022-09-03 15:40:23.641913


In [86]:
### Análise se o pedido de senha foi feito fora do horário do expediente
inicio_expediente = pd.to_datetime(datetime.datetime.now().replace(hour = 7, minute = 0, second = 0, microsecond = 0))
fim_expediente = pd.to_datetime(datetime.datetime.now().replace(hour = 17, minute = 0, second = 0, microsecond = 0))

# horario_atual = pd.to_datetime(datetime.datetime.now())
horario_atual = pd.to_datetime(datetime.datetime.now().replace(hour = 6))

if (horario_atual < inicio_expediente) or (horario_atual > fim_expediente):
    print({"senha": "Fora do Expediênte de Trabalho"})

{'senha': 'Fora do Expediênte de Trabalho'}


In [89]:
print(a)

None


In [84]:
horario_atual

Timestamp('2022-09-03 06:33:51.883243')

In [85]:
inicio_expediente

Timestamp('2022-09-03 07:00:00')

In [ ]:
def ultima_senha(tipo, inicio_expediente):
    df = pd.read_csv("atendimentos.csv")

    ultimo_registro = df[df["Tipo_Senha"] == tipo].tail(1)
    data_ultimo_registro = pd.to_datetime(ultimo_registro["Data_Emissao"]).values[0]

    if data_ultimo_registro < inicio_expediente:
        last_password = 0
    else:
        last_password = int(ultimo_registro["Numeracao"])

    return last_password

In [27]:
df

,ID,Tipo_Senha,Numeracao,Data_Emissao,Guiche,Data_Atendimento
0,1,SP,1,2022-09-03 14:58:23.641908,1.0,2022-09-03 15:08:23.641908
1,2,SE,1,2022-09-03 14:58:23.641909,1.0,2022-09-03 15:08:23.641909
2,3,SG,1,2022-09-03 14:58:23.641910,2.0,2022-09-03 15:08:23.641910
3,4,SP,2,2022-09-03 14:58:23.641911,1.0,2022-09-03 15:08:23.641911
4,5,SE,2,2022-09-03 14:58:23.641912,2.0,2022-09-03 15:08:23.641912
5,6,SP,3,2022-09-03 14:58:23.641913,2.0,2022-09-03 15:08:23.641913
6,7,SG,2,2022-09-03 14:58:23.641914,3.0,2022-09-03 15:08:23.641914
7,8,SG,3,2022-09-03 14:58:23.641915,NaN,NaN
8,9,SP,4,2022-09-03 14:58:23.641916,NaN,NaN
9,10,SG,4,2022-09-03 14:58:23.641917,NaN,NaN


In [26]:
df = df.append({
            'ID' : max_id + 1
            ,'Tipo_Senha' : "SP"
            ,'Numeracao' : 5
            , 'Data_Emissao': datetime.datetime.now()
            } , 
            ignore_index=True)

In [ ]:
df.to_csv("atendimentos.csv")

In [29]:
pd.read_csv('atendimentos.csv').tail()

,ID,Tipo_Senha,Numeracao,Data_Emissao,Guiche,Data_Atendimento
5,6,SP,3,2022-09-03 14:58:23.641913,2.0,2022-09-03 15:08:23.641913
6,7,SG,2,2022-09-03 14:58:23.641914,3.0,2022-09-03 15:08:23.641914
7,8,SG,3,2022-09-03 14:58:23.641915,NaN,NaN
8,9,SP,4,2022-09-03 14:58:23.641916,NaN,NaN
9,10,SG,4,2022-09-03 14:58:23.641917,NaN,NaN


In [74]:
pd.to_datetime(ultimo_registro["Data_Emissao"]).values[0] < pd.to_datetime(inicio_expediente)

TypeError: '<' not supported between instances of 'int' and 'datetime.datetime'

In [ ]:
data_emissao

In [13]:
str(last_password).zfill(3)

'004'